### Data preprocessing 


#### 1. Exploratory analysis & Outlier detection



In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

#### Load data

1. AquaStat 

In [5]:
df = pd.read_csv('clean data/aqua_socec_clean.csv', index_col=0)
df.sample(5)

,country,variable,1998-2002,2003-2007,2008-2012,2013-2017
1387,NIC,Urban population (1000 inhab),2869.477000,3110.511000,3368.137000,3624.780000
2181,ZMB,GDP per capita (current US$/inhab),377.129939,1104.582234,1690.361466,1513.276090
2050,UGA,Human Development Index (HDI) [highest = 1] (-),0.418000,0.457000,0.497000,0.522000
90,AUS,Population density (inhab/km2),2.506242,2.701944,2.958700,3.175807
547,DJI,Rural population with access to safe drinking-...,62.600000,63.800000,64.700000,64.700000


In [6]:
# rename year period to merge with water stress indicators
df.rename(columns = {'country':'Country' ,
                     '1998-2002':'2002', 
                     '2003-2007': '2007', 
                     '2008-2012': '2012',
                     '2013-2017':'2017'}, inplace = True)

2. Water stress data

In [7]:
df_ws = pd.read_csv('clean data/water_stress.csv')
df_ws.sample(5)

,Country,Year,Water stress (MDG),Water use efficiency (SDG),Water stress (SDG)
384,LUX,2007,1.421714,NaN,4.126036
28,AUS,2002,4.219773,NaN,8.347923
32,AUT,2002,4.603604,78.500721,9.883946
79,BOL,2007,0.361280,NaN,1.168967
27,ATG,2017,8.461538,109.839023,8.461538


In [8]:
df_ws["Year"].unique()
df_ws["Year"] = df_ws["Year"].astype("string")
df_ws["Year"] = df_ws["Year"].astype("object")
df_ws["Year"].unique()

array(['2002', '2007', '2012', '2017'], dtype=object)

3. UNICEF dataset

In [10]:
df_unif = pd.read_csv('clean data/unicef_socec_clean.csv', index_col=0)
df_unif.sample(5)


,Indicator,Country,Time,Value
107413,GDP (constant LCU),JOR,2009,2.332862e+10
110919,"PPP conversion factor, private consumption (LC...",IRL,2015,1.000000e+00
111187,GNI per capita (current LCU),ROU,2014,3.342500e+04
50510,"PPP conversion factor, GDP (LCU per internatio...",BFA,2013,2.310800e+02
65376,Population aged 25-64 years (thousands),MDG,2008,6.713000e+03


In [12]:
df_unif.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80784 entries, 0 to 133772
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Indicator  80784 non-null  object 
 1   Country    80784 non-null  object 
 2   Time       80784 non-null  int64  
 3   Value      80784 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.1+ MB


In [13]:
# transform type indicators and slice length name
df["variable"] = df["variable"].astype("string")
df['variable'].unique()
df["variable"] = df['variable'].str.slice(0,35)
df['variable'].unique()

<StringArray>
[      'Rural population (1000 inhab)',       'Urban population (1000 inhab)',
      'Population density (inhab/km2)',  'GDP per capita (current US$/inhab)',
 'Human Development Index (HDI) [high', 'Total population with access to saf',
 'Rural population with access to saf', 'Urban population with access to saf']
Length: 8, dtype: string

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1120 entries, 0 to 2187
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Country   1120 non-null   object 
 1   variable  1120 non-null   string 
 2   2002      1120 non-null   float64
 3   2007      1120 non-null   float64
 4   2012      1120 non-null   float64
 5   2017      1120 non-null   float64
dtypes: float64(4), object(1), string(1)
memory usage: 61.2+ KB


In [ ]:
# transform data aquastat long format (year as attribute)
df_melt = pd.melt(df, id_vars=["Country","variable"], 
        var_name="Year", 
        value_name="value")
df_melt.head(5)

In [ ]:
# transform data aquastat wide format (indicators as attributes)
df_piv = df_melt.pivot_table(values='value', 
                             index=['Year','Country'], 
                             columns='variable',
                             fill_value=0)
df_piv = df_piv.reset_index()

In [ ]:
df_piv.head(5)

In [ ]:
# merge aquastat with water stress. 
# left merge to only keep aquastat countries
df_piv_ws = pd.merge(df_piv, df_ws, on=["Country", "Year"], how='left')
df_piv_ws.sample(5)

In [ ]:
# Distribution per indicator en time period
#variable = df['variable'].unique()

variable = df_piv_ws.columns[2:]

for i in variable:
    plt.figure(figsize=(10,4))
    sns.boxplot(data = df_piv_ws, x = i ,y ='Year', width=0.5)
    plt.title(("Distribution of", i)) 

In [ ]:
# quick check for one time period
df_p = df[['Country','variable','2017']]
# df_p.head(10)

In [ ]:
# df_p.describe()

In [ ]:
df_p = df_p.pivot('Country','variable','2017')
#corr_data = df_p.corr()

In [ ]:
# transform variables (std) before heatmap
scaler = StandardScaler()
# get numeric data

num_d = df_p.select_dtypes(include=['float64'])

# update the cols with their normalized values
df_p[num_d.columns] = scaler.fit_transform(num_d)

In [ ]:
plt.subplots(figsize=(6,.2*(len(df_p.index))))
sns.heatmap(df_p,      
            cbar_kws={'fraction' : 0.08}, # shrink colour bar
            cmap='OrRd', # use orange/red colour map
            linewidth=1)

In [ ]:
sns.clustermap(df_p, 
               linewidth=0.05,
               cmap='OrRd',
               figsize=(9,(.18*(len(df_p.index)))),
               cbar_pos=(1.05, .2, .03, .4))

# Rural & total pop with access to water 
# Rutal & total pop per 1000 habs same: decide which to drop


In [ ]:
sns.pairplot(df_p,kind="reg", plot_kws={'line_kws':{'color':'red'}})

In [ ]:
#sns.pairplot(df_p,kind="reg")
#plot_kws={'line_kws':{'color':'red'}})

### Exploratory analysis AquaStat and Water Stress




In [ ]:
# Exploratory analysis aquastat and ws (one time period)
df_piv_ws_17 = df_piv_ws.loc[df_piv_ws.Year == '2017']
df_piv_ws_17.sample(5)

In [ ]:
df_piv_ws_17.describe()

In [ ]:
df_piv_ws_17.nlargest(5,'Water use efficiency (SDG)')
# looking above boxplot per year Water eff for luxemburg might be an outlier 

In [ ]:
df_piv_ws_17.nlargest(5,'GDP per capita (current US$/inhab)')
# Luxemburg has low population density and largest water use efficiency + highest GDP 

In [ ]:
df_piv_ws_17.nlargest(5,'Water stress (MDG)')

In [ ]:
corr_data = df_piv_ws_17.corr()

In [ ]:
sns.heatmap(corr_data,
           linewidths=.5)

# drop: - Rural or urban pop with access to saf
#       - WS (MDG) or (SDG)
#       - Rural & Urban pop(1000 inhab) 

In [ ]:
df_keep = df_piv_ws_17.loc[:, ~df_piv_ws_17.columns.isin(['Rural population with access to saf',
                                       'Water stress (MDG)',
                                       'Urban population (1000 inhab)'])]
sns.pairplot(df_keep,kind="reg")
#plot_kws={'line_kws':{'color':'red'}})

In [ ]:
# After removing LUX

#g.set(xscale="log", yscale = "log")

df_replot = df_piv_ws_17.loc[~((df_piv_ws_17['Country'] == 'LUX')),:]
g = sns.relplot(x="Population density (inhab/km2)", y='Water use efficiency (SDG)',
               size="Population density (inhab/km2)",
                hue = 'Human Development Index (HDI) [high',
                alpha =0.3,
      #          style="event",
      #          col="country",
                height=8, aspect=.8, 
                #kind="line", 
              sizes=(0,0.5*max(df_replot["Population density (inhab/km2)"])),
                data=df_replot)



In [ ]:
g = sns.relplot(x="GDP per capita (current US$/inhab)", y='Water use efficiency (SDG)',
                size="Population density (inhab/km2)",
                hue = 'Human Development Index (HDI) [high',
                alpha =0.3,
      #         style="event",
      #         col="country",
                height=5, aspect=.8, 
                #kind="line", 
                sizes=(0,0.5*max(df_piv_ws_17["Population density (inhab/km2)"])),
                data=df_piv_ws_17)

3. Exploratory UNICEF dataset

In [15]:
df_unif.describe()

,Time,Value
count,80784.000000,8.078400e+04
mean,2009.510373,1.611890e+13
std,5.764977,6.139415e+14
min,2000.000000,-6.210000e+01
25%,2005.000000,9.000000e+00
50%,2010.000000,3.304000e+03
75%,2014.000000,3.407799e+09
max,2020.000000,1.528323e+17
